In [403]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
%matplotlib inline


In [404]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
submit = pd.read_csv('gender_submission.csv')

In [405]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [406]:
data = train.append(test)

In [407]:
data.reset_index(inplace=True, drop=True)

In [408]:
data['Title1'] = data['Name'].str.split(", ", expand=True)[1]
data['Title1'] = data['Title1'].str.split(".", expand=True)[0]
data['Title2'] = data['Title1'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','the Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],
         ['Miss','Mrs','Miss','Mr','Mr','Mrs','Mrs','Mr','Mr','Mr','Mr','Mr','Mr','Mrs'])

In [409]:
data['Ticket_info'] = data['Ticket'].apply(lambda x : x.replace(".","").replace("/","").strip().split(' ')[0] if not x.isdigit() else 'X')

In [410]:
data['Embarked'] = data['Embarked'].fillna('S')
data['Fare'] = data['Fare'].fillna(data['Fare'].mean())
data["Cabin"] = data['Cabin'].apply(lambda x : str(x)[0] if not pd.isnull(x) else 'NoCabin')

In [411]:
data['Sex'] = data['Sex'].astype('category').cat.codes
data['Embarked'] = data['Embarked'].astype('category').cat.codes
data['Pclass'] = data['Pclass'].astype('category').cat.codes
data['Title2'] = data['Title2'].astype('category').cat.codes
data['Cabin'] = data['Cabin'].astype('category').cat.codes
data['Ticket_info'] = data['Ticket_info'].astype('category').cat.codes

In [412]:
data['Family_Size']=data['SibSp']+data['Parch']

In [413]:
dataAgeNull = data[data["Age"].isnull()]
dataAgeNotNull = data[data["Age"].notnull()]
remove_outlier = dataAgeNotNull[(np.abs(dataAgeNotNull["Fare"]-dataAgeNotNull["Fare"].mean())<(4*dataAgeNotNull["Fare"].std()))|
                      (np.abs(dataAgeNotNull["Family_Size"]-dataAgeNotNull["Family_Size"].mean())<(4*dataAgeNotNull["Family_Size"].std()))                     
                     ]
Model_age = GradientBoostingRegressor(n_estimators=100,random_state=42)
ageColumns = ['Embarked', 'Fare', 'Pclass', 'Sex', 'Family_Size', 'Title2','Cabin','Ticket_info']
Model_age.fit(remove_outlier[ageColumns], remove_outlier["Age"])

ageNullValues = Model_age.predict(X= dataAgeNull[ageColumns])
dataAgeNull.loc[:,"Age"] = ageNullValues
data = dataAgeNull.append(dataAgeNotNull)
data.reset_index(inplace=True, drop=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [414]:
dataTrain = data[pd.notnull(data['Survived'])].sort_values(by=["PassengerId"])
dataTest = data[~pd.notnull(data['Survived'])].sort_values(by=["PassengerId"])

In [415]:
dataTrain = dataTrain[['Survived', 'Age', 'Embarked','Title2' ,'Fare', 'Pclass', 'Sex', 'Family_Size','Ticket_info']]
dataTest = dataTest[  [            'Age', 'Embarked','Title2' ,'Fare', 'Pclass', 'Sex', 'Family_Size','Ticket_info']]

In [416]:
gboost = GradientBoostingClassifier(n_estimators=60, learning_rate=0.08,max_depth=4, random_state=10,max_leaf_nodes=5,
                                   subsample=0.8)

gboost.fit(dataTrain.iloc[:, 1:], dataTrain.iloc[:, 0])


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.08, loss='deviance', max_depth=4,
              max_features=None, max_leaf_nodes=5,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=60, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

In [417]:
gboost.score(dataTrain.iloc[:, 1:], dataTrain.iloc[:, 0])

0.86980920314253651

In [418]:
scores = cross_val_score(gboost, dataTrain.iloc[:, 1:], dataTrain.iloc[:, 0], cv=8)

In [419]:
scores

array([ 0.80357143,  0.80357143,  0.84821429,  0.875     ,  0.79464286,
        0.79279279,  0.80909091,  0.84545455])

In [420]:
pd.concat((pd.DataFrame(dataTrain.iloc[:, 1:].columns, columns = ['variable']), 
           pd.DataFrame(gboost.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

,variable,importance
5,Sex,0.238110
3,Fare,0.232861
0,Age,0.220595
4,Pclass,0.121823
6,Family_Size,0.089951
2,Title2,0.050368
1,Embarked,0.027159
7,Ticket_info,0.019132


In [401]:
model = XGBClassifier(early_stopping_rounds=50, eval_metric="logloss", verbose=True)
model.fit(dataTrain.iloc[:, 1:], dataTrain.iloc[:, 0])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=50, eval_metric='logloss',
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1, verbose=True)

In [402]:
accuracy = accuracy_score(dataTrain.iloc[:, 0],predictions)
accuracy

0.93602693602693599

In [421]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [422]:
model = XGBClassifier()
learning_rate = [0.01, 0.05, 0.8, 0,1, 0.15, 0.3]
max_depth=[2,3,4,5,6,7]
n_estimators=[20,40,60]

param_grid = dict(learning_rate=learning_rate,max_depth=max_depth,n_estimators=n_estimators)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=10)
grid_result = grid_search.fit(dataTrain.iloc[:, 1:], dataTrain.iloc[:, 0])


In [423]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: -0.400990 using {'learning_rate': 0.15, 'max_depth': 4, 'n_estimators': 40}


In [424]:
model = XGBClassifier(eval_metric="auc", learning_rate=0.15,verbose=True,max_depth=3,n_estimators=60)
model.fit(dataTrain.iloc[:, 1:], dataTrain.iloc[:, 0])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=0, learning_rate=0.15,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=60, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1, verbose=True)

In [425]:
y_pred = model.predict(dataTrain.iloc[:, 1:])
accuracy = accuracy_score(dataTrain.iloc[:, 0],y_pred)
accuracy

0.87654320987654322

In [426]:
y_test = model.predict(dataTest)


In [427]:
submit['Survived']=y_test
submit['Survived'] = submit['Survived'].astype(int)
submit.to_csv('submit2.csv', index= False)